In [56]:
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/justinkaufman/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

## Read in data

In [5]:
df=pd.read_csv("/Users/justinkaufman/Stats_101C/Final_Project/Data_Final")
df.drop(columns=['Unnamed: 0'], inplace=True)

In [6]:
df.head()

User_id                  Bus_id  Star  Useful  Cool  Funny  \
0  4hBhtCSgoxkrFgHa4YAD-w  bbEXAEFr4RYHLlZ-HFssTA   5.0       0     0      0   
1  JYYYKt6TdVA4ng9lLcXt_g  SZU9c8V2GuREDN5KgyHFJw   5.0       0     0      0   
2  Uk3X2AypU8AqvcYEVf7s6Q  eL4lyE7LNoXEMvpcJ8WNVw   3.0       5     0      0   
3  LcqNuhqaYt5ekKzaRirmIg  SZU9c8V2GuREDN5KgyHFJw   5.0       2     1      0   
4  gasLVm0KRwrVhPGRcqATjw  CHh0ZFrQcsk4boOItr2Zuw   4.0       1     2      0   

                                              Review State           City  \
0  Great burgers,fries and salad!  Burgers have a...    CA         Goleta   
1  We were a bit weary about trying the Shellfish...    CA  Santa Barbara   
2  This was a tough one!   On the merits of the w...    CA  Santa Barbara   
3  I love trying fresh seafood on piers, wharfs a...    CA  Santa Barbara   
4  I stopped in because I was hungry for some sna...    CA    Carpinteria   

   Bus_Ave_Star  User_Review_count  User_Useful_count  User_Funny_count  \
0           4.0                922               1687               694   
1           4.0                338                800               144   
2           4.0                431               2126              1245   
3           4.0                258                452               125   
4           3.0               1638               2658               933   

   User_Cool_count                                              Elite  \
0             1070                2015,2016,2017,2018,2019,20,20,2021   
1              353  2012,2013,2014,2015,2016,2017,2018,2019,20,20,...   
2             1476  2012,2013,2014,2015,2016,2017,2018,2019,20,20,...   
3              183                                     2015,2016,2017   
4             2297           2014,2015,2016,2017,2018,2019,20,20,2021   

   User_Fans  Users_Ave_Star  
0         51            4.20  
1         30            4.12  
2         76            4.05  
3         37            3.99  
4        105            3.79

## Remove special characters and perform lemmatization

In [8]:
documents = []
X = df.Review
stemmer = WordNetLemmatizer()
for i in range(0, len(X)):
    # Remove all the special characters, like parathesis
    document = re.sub(r'\W', ' ', str(X[i]))
    # remove all single characters: like a, b, c, d
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    # Converting to Lowercase
    document = document.lower()
    # Lemmatization
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    documents.append(document)

In [26]:
vectorizer = CountVectorizer(max_features=1500, min_df=0.1, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents)

## Assign new column with cleaned up reviews

In [12]:
df['review_clean']=documents

In [13]:
df.head()

User_id                  Bus_id  Star  Useful  Cool  Funny  \
0  4hBhtCSgoxkrFgHa4YAD-w  bbEXAEFr4RYHLlZ-HFssTA   5.0       0     0      0   
1  JYYYKt6TdVA4ng9lLcXt_g  SZU9c8V2GuREDN5KgyHFJw   5.0       0     0      0   
2  Uk3X2AypU8AqvcYEVf7s6Q  eL4lyE7LNoXEMvpcJ8WNVw   3.0       5     0      0   
3  LcqNuhqaYt5ekKzaRirmIg  SZU9c8V2GuREDN5KgyHFJw   5.0       2     1      0   
4  gasLVm0KRwrVhPGRcqATjw  CHh0ZFrQcsk4boOItr2Zuw   4.0       1     2      0   

                                              Review State           City  \
0  Great burgers,fries and salad!  Burgers have a...    CA         Goleta   
1  We were a bit weary about trying the Shellfish...    CA  Santa Barbara   
2  This was a tough one!   On the merits of the w...    CA  Santa Barbara   
3  I love trying fresh seafood on piers, wharfs a...    CA  Santa Barbara   
4  I stopped in because I was hungry for some sna...    CA    Carpinteria   

   Bus_Ave_Star  User_Review_count  User_Useful_count  User_Funny_count  \
0           4.0                922               1687               694   
1           4.0                338                800               144   
2           4.0                431               2126              1245   
3           4.0                258                452               125   
4           3.0               1638               2658               933   

   User_Cool_count                                              Elite  \
0             1070                2015,2016,2017,2018,2019,20,20,2021   
1              353  2012,2013,2014,2015,2016,2017,2018,2019,20,20,...   
2             1476  2012,2013,2014,2015,2016,2017,2018,2019,20,20,...   
3              183                                     2015,2016,2017   
4             2297           2014,2015,2016,2017,2018,2019,20,20,2021   

   User_Fans  Users_Ave_Star  \
0         51            4.20   
1         30            4.12   
2         76            4.05   
3         37            3.99   
4        105            3.79   

                                        review_clean  
0  great burger fry and salad burger have hint of...  
1  we were bit weary about trying the shellfish c...  
2  this wa tough one on the merit of the wine fai...  
3  i love trying fresh seafood on pier wharf and ...  
4  i stopped in because wa hungry for some snack ...

## Create df of word frequencies per user review

In [38]:
td = pd.DataFrame(X.todense()) 
td.columns = vectorizer.get_feature_names()

/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [39]:
td

also  always  amazing  area  around  back  bar  barbara  best  better  \
0         0       0        0     0       0     1    0        0     0       0   
1         0       0        1     0       0     0    0        0     0       0   
2         0       0        0     0       0     0    0        0     1       0   
3         1       0        0     0       0     0    0        0     0       1   
4         0       1        0     0       0     0    0        0     0       0   
...     ...     ...      ...   ...     ...   ...  ...      ...   ...     ...   
53840     1       0        0     0       0     0    0        0     0       0   
53841     0       0        0     0       0     1    0        0     0       0   
53842     0       1        0     0       0     0    0        1     1       0   
53843     1       0        0     0       2     2    0        1     0       0   
53844     0       0        0     0       0     0    0        0     0       0   

       ...  thing  think  time  try  two  wa  want  way  well  would  
0      ...      0      0     0    0    0   0     0    0     1      0  
1      ...      0      1     0    1    0   6     0    0     0      1  
2      ...      0      0     3    0    1   1     1    1     0      1  
3      ...      1      0     2    0    0  14     0    0     1      0  
4      ...      0      0     1    0    0   3     0    0     1      0  
...    ...    ...    ...   ...  ...  ...  ..   ...  ...   ...    ...  
53840  ...      1      0     1    3    4   2     3    0     1      0  
53841  ...      0      0     2    0    1   1     0    0     0      0  
53842  ...      0      0     0    0    1   0     0    0     0      0  
53843  ...      0      0     0    0    0   3     1    0     0      0  
53844  ...      0      0     0    0    0   0     1    0     0      0  

[53845 rows x 73 columns]

## Perform K-fold CV

In [43]:
X=td
y=df.Star

In [52]:
lr = LogisticRegression()

In [54]:
cv_score_lr=cross_val_score(lr, X, y, cv=5)

/opt/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

In [55]:
np.mean(np.array(cv_score_lr))

0.48622899062122765

In [58]:
rf=RandomForestClassifier()

In [60]:
cv_score_rf=cross_val_score(rf, X, y, cv=5)

In [61]:
np.mean(np.array(cv_score_rf))

0.46033986442566627